# gebmind.ipynb

Mapas e integración de predicciones.

In [ ]:
import pandas as pd
import joblib
import folium

PROC_CSV = 'googlemaps/data/MAPS_locales_procesado.csv'
MODEL_PATH = 'models/rf_classifier.pkl'

df = pd.read_csv(PROC_CSV)
clf = joblib.load(MODEL_PATH)

feature_cols = ['latitud','longitud','categoria_negocio','dist_city_center_km',
                'density_500m','density_1000m','density_2000m','ratio_500m_2km','cluster_zone'] + list(names)
df['pred_clase'] = clf.predict(df[feature_cols])
df['pred_proba'] = clf.predict_proba(df[feature_cols]).max(axis=1)

city_center = (40.4168, -3.7038)
m = folium.Map(location=city_center, zoom_start=12)
colors = {'baja':'red','media':'orange','alta':'green'}
for _, r in df.iterrows():
    folium.CircleMarker(location=(r.latitud, r.longitud), radius=4,
                        color=colors[r.pred_clase], fill=True,
                        fill_color=colors[r.pred_clase], fill_opacity=0.7,
                        popup=f"ID: {r.id_local}<br>Clase: {r.pred_clase} ({r.pred_proba:.2f})<br>Cat: {r.categoria_negocio}"
                       ).add_to(m)
m